In [18]:
#import shelve
#filename='Hk01Save_28Apri2018.out'
#my_shelf = shelve.open(filename,'n') # 'n' for new

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import jieba
import jieba.posseg
import jieba.analyse

### Data PreProcessing

In [174]:
data = pd.read_csv('offsite-tagging-training-set (1).csv')
test = pd.read_csv('offsite-tagging-test-set (1).csv')
#Read Stop Word
with open('stopword.txt', 'r') as f:
    stopword = []
    for item in f:
        stopword.append(item)

In [175]:
# Check Class Imbalance
# It seems ok, will check the result if imbalance does exist 

data.groupby('tags')['id'].nunique(), data.groupby('tags')['id'].nunique()/data.shape[0]

(tags
 梁振英      929
 美國大選     842
 足球      2123
 Name: id, dtype: int64, tags
 梁振英     0.238572
 美國大選    0.216230
 足球      0.545198
 Name: id, dtype: float64)

In [176]:
# Combine file first and will be separate later on
data['type']=1
test['type']=0

In [177]:
# Check Shape
data.shape,test.shape

((3894, 4), (974, 4))

In [178]:
# Merge
datacon = pd.concat([data,test])

In [179]:
#Remove HTML-Tag data
def cleanhtmltag(sentence): 
    clean = re.compile('<.*?>')
    cleantext = re.sub(clean, ' ', sentence)
    return cleantext

In [180]:
#Remove punctuation
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|（|）|。|」|「|【|】|\'|，|)|(|\|/]',r' ',cleaned)
    cleaned =re.sub('[\s+]', '', cleaned)#space
    return  cleaned

In [181]:
#Clear Text
import re
final_string=[]
str1=' '
i=0
s=' '

for sent in datacon['text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtmltag(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words not in stopword):
                filtered_sentence.append(cleaned_words)
            else:
                continue 
    #print('-----------------------------------------------')
    #print(filtered_sentence)
    str1 = s.join(filtered_sentence)
    #print(str1)
    final_string.append(str1)
    i+=1

In [182]:
datacon['ClearText'] = final_string

In [183]:
# Tokenization with different methodology - jieba.analyse.extract_tags and TextRank
import jieba.analyse
tokenTextET =[]
tokenTextRK=[]

tfidf = jieba.analyse.extract_tags
textrank = jieba.analyse.textrank

for i in datacon['ClearText']:
    tagsET = jieba.analyse.extract_tags(i, topK=100 )#,withWeight=True)
    tagsET = ' '.join(tagsET)
    tokenTextET.append(tagsET)
     
#    tagsRK  = textrank(i)
#    tagsRK = ' '.join(tagsRK)
#    tokenTextRK.append(tagsRK)

In [184]:
datacon['tokenTextET'] = tokenTextET
#datacon['tokenTextTO'] = tokenTextRK

In [185]:
datacon.head(1)

,id,tags,text,type,ClearText,tokenTextET
0,3443,足球,利物浦重賽擊敗乙組仔 英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組...,1,利物浦重賽擊敗乙組仔 英足盃過關 英格蘭足總盃第三圈今晨重賽貴為英超勁旅的利物浦上場被乙組仔...,第三圈 對手 擊敗 組仔 重賽 利物浦 埃克斯 迪西 舒爾奧祖及 上場 尷尬 主場 回勇 開...


In [186]:
#Separate dataset
test =datacon[datacon['type']==0]
data=datacon[datacon['type']==1]

In [187]:
# tdidf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer=CountVectorizer()
transformer=TfidfTransformer()
tfidf = TfidfVectorizer()

dataRkET= transformer.fit_transform(vectorizer.fit_transform(data['tokenTextET']))
dataRkETword=vectorizer.get_feature_names()
dataRkETweight=dataRkET.toarray()

testRkET=transformer.transform(vectorizer.transform(test['tokenTextET']))
#testRk = tfidf.fit_transform(test['ClearText']) 
testRkETword=vectorizer.get_feature_names()
testRkETweight=testRkET.toarray()

#dataRK= transformer.fit_transform(vectorizer.fit_transform(data['tokenTextTO']))
#dataRkword=vectorizer.get_feature_names()
#dataRkweight=dataRK.toarray()

#testRK= transformer.transform(vectorizer.fit_transform(data['tokenTextTO']))
#testRkETword=vectorizer.get_feature_names()
#testRkETweight=testRK.toarray()

In [188]:
my_shelf.close()

#To restore
import shelve
filename='Hk01Save_28Apri2018.out'
my_shelf = shelve.open(filename)
for key in my_shelf:
    globals()[key]=my_shelf[key]
my_shelf.close()

In [189]:
# Replace Tags to numberic number 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_label = le.fit_transform(data['tags'])
test_label = le.transform(test['tags'])

In [190]:
# Dimension reduction for jieba.analyse.extract_tags
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=3000, n_iter=25, random_state=1234)
dataRktd= svd.fit_transform(dataRkET)
explained_variance = svd.explained_variance_ratio_.sum()
print(" : Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
   
testRktd= svd.transform(testRkET)
explained_variance = svd.explained_variance_ratio_.sum()
print(" : Explained variance of the SVD step: {}%".format(int(explained_variance * 100))) 

 : Explained variance of the SVD step: 87%
 : Explained variance of the SVD step: 87%


In [191]:
# Dimension reduction for jieba textrank
#svdrk = TruncatedSVD(n_components=3000, n_iter=25, random_state=1234)
#dataRktr= svdrk.fit_transform(dataRK)
#explained_variance = svdrk.explained_variance_ratio_.sum()
#print(" : Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
   
#testRktr= svdrk.transform(testRK)
#explained_variance = svdrk.explained_variance_ratio_.sum()
#print(" : Explained variance of the SVD step: {}%".format(int(explained_variance * 100))) 

### XGBoost Training for jieba.analyse.extract_tags - dataRktd and testRktd

In [192]:
Y_train=data['tags']
Y_valid=test['tags']

In [193]:
# Split data for validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataRktd, Y_train, test_size=0.3)

In [194]:
import xgboost as xgb
from xgboost import XGBClassifier  
model_xgbt = XGBClassifier(
 colsample_bytree=0.9,
 gamma=0.75  ,
 max_depth=11,   
 learning_rate=0.2, 
 nthread =-1,
 subsample=0.8,
 min_child_weight=14,   
 seed=1234,
 n_estimators=100
)
model_xgbt.fit(X_train,y_train,eval_metric='auc')   

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.75, learning_rate=0.2, max_delta_step=0, max_depth=11,
       min_child_weight=14, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=0.8)

In [195]:
from sklearn import metrics  
y_true, y_pred = y_test, model_xgbt.predict(X_test)  
print ("Accuracy : %.4g" % metrics.accuracy_score(y_true, y_pred)  )

Accuracy : 0.9923


In [196]:
from sklearn.metrics import accuracy_score ,classification_report,confusion_matrix
acc = accuracy_score(y_test, y_pred)
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        梁振英       1.00      0.98      0.99       261
       美國大選       1.00      0.99      0.99       249
         足球       0.99      1.00      0.99       659

avg / total       0.99      0.99      0.99      1169



In [197]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(model_xgbt, X_test, y_test, cv=10, scoring='accuracy')
np.mean(scores)

0.9871497584541062

In [198]:
y_valid=test['tags']

In [199]:
y_true, y_pred = y_valid, model_xgbt.predict(testRktd)  
print ("Accuracy : %.4g" % metrics.accuracy_score(y_true, y_pred)  )

Accuracy : 0.9836


In [200]:
testbk=test

In [201]:
test = test.drop(['tags','type','ClearText','tokenTextET'],axis=1)

In [202]:
test['tags']=y_pred

In [203]:
test.to_csv('HK01_Q3_XGboost2.csv', sep='\t')

In [172]:
test.head(200)

,id,text,tags
0,6,南華添鋒力 簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix ...,足球
1,128,如果大學$0捐款 科大嶺南將蝕過千萬元 據now新聞台報道，身兼8大校監的特首梁振英曾以大學...,梁振英
2,136,英超最強火力對碰 雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 5」只剩兩隊名列積分榜前5...,足球
3,213,【01球評】膺半程冠軍 阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環...,足球
4,658,【書商失蹤】梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李...,梁振英
5,700,【施政盤點】三份施政報告 僅一半政策達標 行政長官梁振英即將公布任內第四份施政報告，《香港0...,梁振英
6,729,【施政盤點】「治港絕招」 設19委員會 空談多實務少 行政長官梁振英上任3年多，先後成立多個...,梁振英
7,837,高普首簽 「新馬迪」來季投紅軍 利物浦傷兵滿營及戰績不穩，主帥高普仍不忘投資未來，昨以510...,足球
8,1037,「最潮主帥」鬥利物浦：我已領先1：0 英乙球隊埃克塞特在明晨的足總盃於主場迎戰利物浦，雖然越...,足球
9,1095,紅軍超殘陣逼和英乙隊 高普：負擔不起重賽 逾十名球員受傷的利物浦，今晨在足總盃第三圈以大部份...,足球


### XGBoost Training for jieba textrank -dataRktr and testRktr

In [63]:
#X_train, X_test, y_train, y_test = train_test_split(dataRktr, Y_train, test_size=0.3)
#model_xgbt.fit(X_train,y_train,eval_metric='auc')   

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.75, learning_rate=0.2, max_delta_step=0, max_depth=11,
       min_child_weight=14, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=0.8)

In [64]:
#y_true, y_pred = y_test, model_xgbt.predict(X_test)  
#print ("Accuracy : %.4g" % metrics.accuracy_score(y_true, y_pred)  )

Accuracy : 0.982


In [65]:
#y_true, y_pred = y_valid, model_xgbt.predict(testRktd)  
#print ("Accuracy : %.4g" % metrics.accuracy_score(y_true, y_pred)  )

Accuracy : 0.5246


### Conclusion

#### Xgboost with  jieba.analyse.extract_tags for tagging seems better than TextRank 
#### Alternative, search parameter with gaussian optimization would be another option